In [1]:
import sys 

from vmtk import pypes
from vmtk import vmtkscripts

**From original code**

`mast_dir = ' D:/Hartmann/PythonAll/seal_whiskers/Har20191023_harbor3_jig1/' `

`dir_1 = mast_dir + 'saved/IMG-0002-00001.dcm ' ###raw dcm stack`

`dir_2 = mast_dir + 'image_volume_voi.vti ' ###image: one whisker`

`dir_5 = mast_dir + 'image_volume_voi_v2.vti ' ###image: many whisker`

`dir_3 = mast_dir + 'mc_surface.vtp ' ### surface`

`dir_4 = mast_dir + 'level_sets.vti ' ###image `

`arg_1 = 'vmtkimagereader -ifile' + dir_1 + '--pipe vmtkimagevoiselector -ofile' +  dir_5`

`arg_3 = 'vmtklevelsetsegmentation -ifile' + dir_5 + '-ofile' + dir_4`

`arg_2 = 'vmtkmarchingcubes -ifile' +  dir_5 +'-l 5.0 -ofile' +  dir_3 + '--pipe vmtksurfaceviewer'`

`myArguments = 'vmtkmarchingcubes -ifile ' + dir_2 + ' -l 5 --pipe vmtksurfaceviewer'`



`#myPype = pypes.PypeRun(myArguments)`

In [5]:
mast_dir = ' C:/Users/corma/Desktop/HLAB_seals/whiskers_temp/d_Har20190607_long_whiskers_68um/'
dir_1 = mast_dir + 'Har20190607_long_whiskers_68um_dcm/IMG-0003-00001.dcm ' ###raw dcm stack
dir_2 = mast_dir + 'image_volume_voi.vti ' ###image: one whisker
dir_5 = mast_dir + 'image_volume_voi_v2.vti ' ###image: many whisker
dir_3 = mast_dir + 'mc_surface.vtp ' ### surface
dir_3_less = mast_dir + 'mc_surf_less.vtp '
dir_4 = mast_dir + 'level_sets.vti ' ###image 
dir_6 = mast_dir + 'mc_surf_smoothed.vtp ' ###smoothed surface
dir_6_extra = mast_dir + 'mc_surf_super_smoothed.vtp ' ###extra smoothed surface
dir_7 = mast_dir + 'sb_surf.vtp ' ###surface subdivision
dir_8 = mast_dir + 'mc_surf_clip.vtp ' ###surface with endcaps clipped off
dir_9 = mast_dir + 'smoothed_surf_centerlines.vtp ' ###centerlines of smoothed surface
dir_10 = mast_dir + 'mc_surf_out.stl ' ###output surface for Geomagic
dir_11 = mast_dir + 'isol_surf.vtp ' ###input surface from Geomagic/SW

arg_1 = 'vmtkimagereader -ifile' + dir_1 + '--pipe vmtkimagevoiselector -ofile' +  dir_5
arg_2 = 'vmtkmarchingcubes -ifile' +  dir_5 +'-l -500.0 -ofile' +  dir_3 + '--pipe vmtksurfaceviewer' ###default 5.0, originally arg_2
arg_2_less =  'vmtkmarchingcubes -ifile' +  dir_5 +'-l -250.0 -ofile' +  dir_3_less + '--pipe vmtksurfaceviewer' ###less noise, use for centerlines calcs
arg_3 = 'vmtksurfacesmoothing -ifile' + dir_3_less + '-method "laplace" -relaxation 0.3 -iterations 50 -ofile' + dir_6_extra ###laplace smoothing method, relaxation .07 is good
###arg_3 = 'vmtksurfacesmoothing -ifile' + dir_3 + '-passband 0.1 -iterations 250 -ofile' + dir_6_extra ###taubin smoothing, originally arg_4
arg_4 =  'vmtksurfacereader -ifile' + dir_3_less  + '--pipe vmtksurfacesmoothing -method "laplace" -relaxation 0.3  -iterations 50  --pipe vmtkrenderer --pipe vmtksurfaceviewer -display 0 --pipe vmtksurfaceviewer -i @vmtksurfacereader.o -color 1 0 0 -display 1' ###using laplace smoothing
###arg_4 =  'vmtksurfacereader -ifile' + dir_3  + '--pipe vmtksurfacesmoothing -iterations 250 -passband 0.1 --pipe vmtkrenderer --pipe vmtksurfaceviewer -display 0 --pipe vmtksurfaceviewer -i @vmtksurfacereader.o -color 1 0 0 -display 1' ###taubin smoothing, originally arg_5
arg_5_opt1 = 'vmtksurfacesubdivision -ifile' + dir_6_extra + '-ofile' + dir_7 + '-method butterfly'###increases number of triangles on surface, if used then change dir_6 in arg_6 to dir_7
arg_5_opt2 = 'vmtksurfaceclipper -ifile' + dir_6_extra + '-ofile' + dir_8 
arg_6 = 'vmtkcenterlines -seedselector openprofiles -ifile' + dir_8 + '-ofile' + dir_9

arg_prepping = 'vmtksurfaceclipper -ifile' + dir_3 + '--pipe vmtksurfacewriter -ofile' + dir_10 
arg_7 = 'vmtksurfacewriter -ifile' + dir_3 + '-ofile' + dir_10
arg_8 = 'vmtksurfacereader -ifile' + dir_10 + ' -ofile' + dir_11

arg_help = 'vmtkcenterlines --help' ###replace 'vmtkcenterlines' with any function you want to know about, faster than googling
#vmtksurfaceconnectivity -ifile airway.vtp --pipe vmtksurfacesmoothing -passband 0.01 -iterations 30 --pipe vmtkcenterlines -ofile airway_centerlines.vtp
    

 
arg_1_opt = 'vmtklevelsetsegmentation -ifile' + dir_5 + '-ofile' + dir_4 ###originally arg_3
arg_2_opt = 'vmtkmarchingcubes -ifile ' + dir_2 + ' -l 5 --pipe vmtksurfaceviewer'###vestigial part of original code, 'myArguments'

#run args1-5 sequentially to automatically segment and extract centerlines
#run arg_1_opt for colliding fronts initialization
#myPype = pypes.PypeRun(myArguments)

In [6]:
myPype = pypes.PypeRun(arg_prepping)


Automatic piping vmtksurfaceclipper
Parsing options vmtksurfaceclipper
    SurfaceInputFileName = C:/Users/corma/Desktop/HLAB_seals/whiskers_temp/d_Har20190607_long_whiskers_68um/mc_surface.vtp
Explicit piping vmtksurfaceclipper
Input vmtksurfaceclipper members:
    Id = 0
    Disabled = 0
    Surface = None
    SurfaceInputFileName = C:/Users/corma/Desktop/HLAB_seals/whiskers_temp/d_Har20190607_long_whiskers_68um/mc_surface.vtp
    WidgetType = box
    Transform = None
    CleanOutput = 1
    InsideOut = 0
    Interactive = 1
    ClipArrayName = None
    ClipValue = 0.0
    vmtkRenderer = None
    SurfaceOutputFileName = 
    ClippedSurfaceOutputFileName = 
    CutLinesOutputFileName = 
Reading VTK XML surface file.
Executing vmtksurfaceclipper ...
Quit renderer
Done executing vmtksurfaceclipper.
Output vmtksurfaceclipper members:
    Id = 0
    Surface = vtkPolyData
    ClippedSurface = vtkPolyData
    CutLines = vtkPolyData
    Transform = vtkTransform

Automatic piping vmtksurface

In [ ]:
myPype = pypes.PypeRun(arg_help)

In [ ]:
mySurface = myPype.GetScriptObject('vmtkmarchingcubes','0').Surface

mySmoother = vmtkscripts.vmtkSurfaceSmoothing()
mySmoother.Surface = mySurface
mySmoother.PassBand = 0.1
mySmoother.NumberOfIterations = 30
mySmoother.Execute()

In [ ]:
myWriter = vmtkscripts.vmtkSurfaceWriter()
myWriter.Surface = mySmoother.Surface
myWriter.OutputFileName = mast_dir + 'mysurface.vtp'
myWriter.Execute()

In [ ]:
customscript = 'customScript'

class customScript(pypes.pypeScript):
    def __init__(self):
        pypes.pypeScript.__init__(self)
        self.Image = None
        self.Surface = None
        self.Levels=[]
        self.SetScriptName('customScript')
        self.SetScriptDoc('here goes a description')
        self.SetInputMembers([
            ['Image','i','vtkImageData',1,'','the input image','vmtkimagereader'],
            ['Levels','levels','float',-1,'','graylevels to generate the isosurface at'],
            ])
        self.SetOutputMembers([
            ['Surface','o','vtkPolyData',1,'','the output surface','vmtksurfacewriter']
            ])
    def Execute(self): 
        if self.Image == None:
            self.PrintError('Error: No Image.')
        if self.Levels == []:
            self.PrintError('Error: No Levels')
        self.marchingCubes = vmtkscripts.vmtkMarchingCubes()
        self.marchingCubes.Image = self.Image
        self.marchingCubes.Connectivity = 1
        self.vmtkRenderer = vmtkscripts.vmtkRenderer()
        self.vmtkRenderer.Initialize()
        self.SurfaceViewer = vmtkscripts.vmtkSurfaceViewer()
        self.SurfaceViewer.vmtkRenderer = self.vmtkRenderer
        for level in self.Levels:
            self.marchingCubes.Level = level
            self.marchingCubes.Execute()
            self.Surface = self.marchingCubes.Surface
            self.SurfaceViewer.Surface = self.Surface
            self.SurfaceViewer.BuildView()     
    pass

if __name__=='__main__':
    main = pypes.pypeMain()
    main.Arguments = sys.argv
    main.Execute()

In [ ]:
     vmtkcenterlines -seedselector openprofiles -ifile tubeExtended.stl -ofile centerlines.vtp  --pipe vmtksurfacewriter -ifile centerlines.vtp -ofile centerlines.vtk